In [8]:
import sys
sys.path.append("../..")

In [9]:
from prettytable import PrettyTable, ALL
from textwrap import fill
from msprof_analyze.advisor.interface.interface import Interface

In [10]:
profiling_path = "YOUR PROFILING PATH"
interface = Interface(profiling_path=profiling_path)

### 融合算子API识别

指定profiling路径后，可以自动识别其中包含的融合算子并给出对应的torch_npu api和需要修改的代码堆栈。基于给定堆栈可以快速定位到需要修改的代码段，替换torch_npu api后，能够减少pytorch侧的小算子的下发，进而提升模型训练速度。

In [ ]:
timeline_fusion_ops_result = interface.get_result("schedule", "timeline_fusion_ops")

In [12]:
display_column_num = 3
problems = timeline_fusion_ops_result.get("问题综述")
problem_table = PrettyTable(problems.get("headers")[:display_column_num])
for row in problems.get("data"):
    for i in range(len(row)):
        row[i] = fill(str(row[i]), width=80)
    problem_table.add_row(row[:display_column_num])

display(problem_table)

category,description,suggestion
亲和API接口,目前运行环境版本为cann-8.0.0和torch-2.1.0，发现有2个api接口可以替换。,1. 请根据子表'Affinity training api'替换训练api接口


如下所示，存在亲和优化器和梯度裁剪两个可替换的torch_npu api，并给出了具体的堆栈。

In [13]:
fusion_ops_api = timeline_fusion_ops_result.get("亲和API接口")
if fusion_ops_api:
    fusion_ops_api_table = PrettyTable(fusion_ops_api.get("headers"))

    for row in fusion_ops_api.get("data"):
        for i in range(len(row)):
            row[i] = fill(str(row[i]), width=80)
        fusion_ops_api_table.add_row(row)

    fusion_ops_api_table.hrules = ALL
    display(fusion_ops_api_table)

Affinity API,Code stacks,Stack called counts
torch_npu.npu_confusion_transpose,/usr/local/lib/python3.8/site-packages/torch/_tensor.py(1000): __iter__;/opt/tiger/janus/janus/megatron/experts.py(2665): backward;/usr/local/lib/python3.8/site-packages/torch/autograd/function.py(288): apply,4
torch_npu.npu_confusion_transpose,/home/tiger/.local/lib/python3.8/site-packages/mindspeed/ops/gmm.py(35):forward; /usr/local/lib/python3.8/site-packages/torch/autograd/function.py(539): apply;/home/tiger/.local/lib/python3.8/site-packages/mindspeed/ops/gmm.py(90):_npu_gmm; /usr/local/lib/python3.8/site-packages/torch/_ops.py(692): __call__;/home/tiger/.local/lib/python3.8/site-packages/mindspeed/ops/gmm.py(94):npu_gmm; /opt/tiger/janus/janus/megatron/experts.py(2667): backward;/usr/local/lib/python3.8/site-packages/torch/autograd/function.py(288): apply,3
torch_npu.npu_confusion_transpose,/usr/local/lib/python3.8/site-packages/torch/_tensor.py(1000): __iter__;/opt/tiger/janus/janus/megatron/experts.py(2389): native_moe_forward;/opt/tiger/janus/janus/megatron/experts.py(2404): forward;/usr/local/lib/python3.8/site-packages/torch/autograd/function.py(539): apply;/opt/tiger/janus/janus/megatron/experts.py(2330): forward;/usr/local/lib/python3.8/site-packages/torch/nn/modules/module.py(1527):_call_impl; /usr/local/lib/python3.8/site-packages/torch/nn/modules/module.py(1518): _wrapped_call_impl;/opt/tiger/janus/janus/megatron/lego_moe_layer.py(418): forward_;/opt/tiger/janus/janus/megatron/lego_moe_layer.py(477): forward;/usr/local/lib/python3.8/site-packages/torch/nn/modules/module.py(1527):_call_impl; /usr/local/lib/python3.8/site-packages/torch/nn/modules/module.py(1518): _wrapped_call_impl;/opt/tiger/mariana/mariana/models/layers/moe.py(468): forward;/usr/local/lib/python3.8/site-packages/torch/nn/modules/module.py(1527):_call_impl; /usr/local/lib/python3.8/site-packages/torch/nn/modules/module.py(1518): _wrapped_call_impl;/opt/tiger/mariana/mariana/models/layers/transformer.py(735): _forward;/opt/tiger/mariana/mariana/models/layers/transformer.py(334): forward;/usr/local/lib/python3.8/site-packages/torch/nn/modules/module.py(1527):_call_impl; /usr/local/lib/python3.8/site-packages/torch/nn/modules/module.py(1518): _wrapped_call_impl;/opt/tiger/mariana/mariana/models/text/transformer.py(509): forward;/usr/local/lib/python3.8/site-packages/torch/nn/modules/module.py(1527):_call_impl; /usr/local/lib/python3.8/site-packages/torch/nn/modules/module.py(1518): _wrapped_call_impl;/opt/tiger/mariana/mariana/models/text/gpt2_megatron.py(204): forward;/usr/local/lib/python3.8/site-packages/torch/nn/modules/module.py(1527):_call_impl; /usr/local/lib/python3.8/site-packages/torch/nn/modules/module.py(1518): _wrapped_call_impl;/opt/tiger/mariana/mariana/models/text/gpt2_megatron.py(445): forward;/usr/local/lib/python3.8/site-packages/torch/nn/modules/module.py(1527):_call_impl; /usr/local/lib/python3.8/site-packages/torch/nn/modules/module.py(1518): _wrapped_call_impl;/opt/tiger/Megatron-LM/megatron/model/module.py(286): forward;/usr/local/lib/python3.8/site-packages/torch/nn/modules/module.py(1527):_call_impl; /usr/local/lib/python3.8/site-packages/torch/nn/modules/module.py(1518): _wrapped_call_impl;/opt/tiger/Megatron-LM/megatron/model/distributed.py(230): forward;/usr/local/lib/python3.8/site-packages/torch/nn/modules/module.py(1527):_call_impl; /usr/local/lib/python3.8/site-packages/torch/nn/modules/module.py(1518): _wrapped_call_impl;tasks/gpt2/unsup/model.py(615): forward; tasks/gpt2/unsup/model.py(881):forward_step; /opt/tiger/Megatron-LM/megatron/schedules.py(305): forward_step;/opt/tiger/Megatron-LM/megatron/schedules.py(461):forward_backward_no_pipelining; tasks/gpt2/unsup/model.py(961):_megatron_fwd_bwd_function; tasks/gpt2/unsup/model.py(1246): training_step;/opt/tiger/cruise/cruise/module/wrapper.py(28): forward;/usr/local/lib/python3.8/site-packages/torch/nn/modules/module.py(1527):_call_impl; /usr/local/lib/python3.8/site-packages/torch/nn